## 🚀 LOAD LIBARIES

In [ ]:
!pip install textstat
!pip install spacy
!pip install wordfreq
!pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusol

In [ ]:
import gdown
import pandas as pd
from textstat import textstat
import os
import spacy
from nltk.corpus import brown
import nltk
from nltk.tokenize import word_tokenize
from wordfreq import word_frequency
import numpy as np
from keybert import KeyBERT
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
nlp = spacy.load('en_core_web_sm')
nltk.download('brown')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

### 📚 LOAD DATASETS

AI Responses

In [ ]:
df_ai_responses = pd.read_csv('/interviews/ai-human/ai-interviewing-responses.csv')
df_ai_responses['Interviewer'] = 'ai'  # we add an identifier to the dataset

Downloading...
From: https://drive.google.com/uc?id=11WMQXU1nOvxrwasChnDthXZAK9lsbX_y
To: /content/ai interviewing-responses.csv
100% 151k/151k [00:00<00:00, 8.04MB/s]


Human Responses

In [ ]:
filepaths = [  # ID ohne Nachnamen
    '/interviews/human-human/interview-transcriped_i4.csv',
    '/interviews/human-human/interview-transcriped_i5.csv',
    '/interviews/human-human/interview-transcriped_i3.csv',
    '/interviews/human-human/interview-transcriped_i2.csv',
    '/interviews/human-human/interview-transcriped_i1s.csv'
]

dfs = [
    pd.read_csv(filepath).rename(columns={'Text': 'MessageContent', 'In': 'Timestamp', 'Role': 'Interlocutor'}).assign(source_file=os.path.splitext(os.path.basename(filepath))[0])
    for filepath in filepaths
]

combined_df = pd.concat(dfs, ignore_index=True)
combined_df['Interviewer'] = 'human'  # we add an identifier to the dataset

Downloading...
From: https://drive.google.com/uc?id=16u3G-fYBGZNtRSche6M4t4QPOBiKKeqJ
To: /content/human_interview_Schuttpelz.csv
100% 13.8k/13.8k [00:00<00:00, 22.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1y-zsDdM1p7_OJRZa4_qEQzfYIjY70Q4W
To: /content/human_interview_Niethammer.csv
100% 12.9k/12.9k [00:00<00:00, 21.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-t6Y3S_8RPn3dEGeMWHN21nbTkNAPUFA
To: /content/human_interview_LoreBueno.csv
100% 10.3k/10.3k [00:00<00:00, 20.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1m43x5IjuMJ-ANH5Ta8mH-FesqxAkeYiW
To: /content/human_interview_Kreider.csv
100% 11.5k/11.5k [00:00<00:00, 33.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-0VmPa6CegfIrluoBLe4t5FDrT14i_d2
To: /content/human_interview_Fuchs.csv
100% 8.99k/8.99k [00:00<00:00, 26.8MB/s]


In [ ]:
combined_df['Word Count'] = combined_df['MessageContent'].apply(lambda x: len(x.split()))
df_ai_responses['Word Count'] = df_ai_responses['MessageContent'].apply(lambda x: len(x.split()))

In [ ]:
(combined_df
 .query("source_file == 'interview-transcriped_i4'")
 .head(5)
)

,Interlocutor,Timestamp,MessageContent,source_file,Interviewer,Word Count
0,Interviewer,00:00:04,Before we start with the questions on society ...,human_interview_Niethammer,human,24
1,Respondent,00:00:11,I am in a breakout room two.,human_interview_Niethammer,human,7
2,Interviewer,00:00:14,"Okay, thanks. Perfect. Let's start. Please not...",human_interview_Niethammer,human,144
3,Respondent,00:01:06,I think the first thing you might think is lik...,human_interview_Niethammer,human,110
4,Interviewer,00:01:48,Okay. Interesting. All right. Can you think of...,human_interview_Niethammer,human,33


## 🔎 SCORES

We can calculate a wide variety of "readability scores" and analyze the complexity of the responses.

In [ ]:
def readability_scores(respones):
    scores = {
        'Flesch Reading Ease': textstat.flesch_reading_ease(respones),
        'SMOG Index': textstat.smog_index(respones),
        'Flesch-Kincaid Grade Level': textstat.flesch_kincaid_grade(respones),
        'Coleman-Liau Index': textstat.coleman_liau_index(respones),
        'Automated Readability Index': textstat.automated_readability_index(respones),
        'Dale-Chall Readability Score': textstat.dale_chall_readability_score(respones),
        'Difficult Words': textstat.difficult_words(respones),
        'Linsear Write Formula': textstat.linsear_write_formula(respones),
        'Gunning Fog': textstat.gunning_fog(respones)
    }
    return scores

In addition, we include various descriptive metrics.

In [ ]:
def descriptive_analysis(df):
  df['Length'] = df['MessageContent'].apply(lambda x: len(x))  # number of chars
  df['Word Count'] = df['MessageContent'].apply(lambda x: len([token.text for token in nlp(x)]))  # number of words (will be updated to integrate the SpaCy word count)
  df['Unique Words'] = df['MessageContent'].apply(lambda x: len(set([token.text for token in nlp(x)])))  # number of unique words/ set (will be updated to integrate the SpaCy word count)
  df['Type-Token-Ratio'] = df['Unique Words'] / df['Word Count']  # ratio of unique words to total words (will be updated to integrate the SpaCy word count)
  df['Flesch Reading Ease'] = df['MessageContent'].apply(lambda x: textstat.flesch_reading_ease(x))  # readability score (we can choose from available scores) - NOTE: we can also use any others
  ## warum kann der Score hier negativ sein?

  return df

We can now apply the defined `descriptive_analysis()` function to the human and ai interview-based responses.

In [ ]:
df_ai_responses = descriptive_analysis(df_ai_responses)
combined_df = descriptive_analysis(combined_df)

df_ai_responses.head()

,StepID,ThreadID,StepURL,ParticipantID,MessageID,Interlocutor,MessageContent,Timestamp,Interviewer,Word Count,Length,Unique Words,Type-Token-Ratio,Flesch Reading Ease
0,273e7fde-1824-4801-b8dc-8acb7e160671,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,1,assistant,Welcome to our in-depth interview about democr...,2024-03-28 14:32:19,ai,24,120,18,0.750000,65.08
1,6ce3c664-be62-4c39-8a32-beb10dbb4a0f,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,2,human,yes,2024-03-28 14:33:32,ai,1,3,1,1.000000,121.22
2,241d3de5-e09b-4db3-a769-cbe8269f156a,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,3,assistant,Before we start with the questions on society ...,2024-03-28 14:33:36,ai,26,133,24,0.923077,64.04
3,660e7b54-8585-45c5-9303-881df97f4562,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,4,human,7,2024-03-28 14:33:54,ai,1,1,1,1.000000,121.22
4,cd931ee1-01f5-4936-b02f-df6bdc36dba5,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,5,assistant,Thank you. Let’s start. Please note that there...,2024-03-28 14:34:07,ai,165,798,95,0.575758,73.78


## ADDITIONAL SCORES (OPTIONAL)

https://onlinelibrary.wiley.com/doi/full/10.1111/ajps.12423 "Measuring and Explaining Political Sophistication through Textual Complexity" schlägt noch verschiedene Metriken vor

In [ ]:
def calculate_pos(df):
    df['POS'] = df['MessageContent'].apply(lambda sentence: calculate_pos_proportions(sentence))
    return df

In [ ]:
def calculate_pos_proportions(text):
    doc = nlp(text)
    num_tokens = len(doc)

    pos_counts = {
        "NOUN": 0,
        "VERB": 0,
        "ADJ": 0,
        "ADV": 0,
        "SUBORDINATE_CLAUSES": 0
    }

    for token in doc:
        if token.pos_ in pos_counts:
            pos_counts[token.pos_] += 1

    for sent in doc.sents:
        if any(token.dep_ == "mark" for token in sent):
            pos_counts["SUBORDINATE_CLAUSES"] += 1

    return {
        "Nouns": pos_counts["NOUN"] / num_tokens,
        "Verbs": pos_counts["VERB"] / num_tokens,
        "Adjectives": pos_counts["ADJ"] / num_tokens,
        "Adverbs": pos_counts["ADV"] / num_tokens,
        "Average": pos_counts["SUBORDINATE_CLAUSES"] / len(list(doc.sents))
    }

In [ ]:
calculate_pos(df_ai_responses)

,StepID,ThreadID,StepURL,ParticipantID,MessageID,Interlocutor,MessageContent,Timestamp,Interviewer,Word Count,Length,Unique Words,Type-Token-Ratio,Flesch Reading Ease,POS,Rare Words,Automated Readability Index
0,273e7fde-1824-4801-b8dc-8acb7e160671,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,1,assistant,Welcome to our in-depth interview about democr...,2024-03-28 14:32:19,ai,19,120,16,0.842105,65.08,"{'Nouns': 0.16666666666666666, 'Verbs': 0.1666...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",7.0
1,6ce3c664-be62-4c39-8a32-beb10dbb4a0f,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,2,human,yes,2024-03-28 14:33:32,ai,1,3,1,1.000000,121.22,"{'Nouns': 0.0, 'Verbs': 0.0, 'Adjectives': 0.0...","{'Brown Corpus Baseline Min': 0.000316, 'Brown...",-6.9
2,241d3de5-e09b-4db3-a769-cbe8269f156a,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,3,assistant,Before we start with the questions on society ...,2024-03-28 14:33:36,ai,24,133,22,0.916667,64.04,"{'Nouns': 0.19230769230769232, 'Verbs': 0.0769...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",12.1
3,660e7b54-8585-45c5-9303-881df97f4562,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,4,human,7,2024-03-28 14:33:54,ai,1,1,1,1.000000,121.22,"{'Nouns': 0.0, 'Verbs': 0.0, 'Adjectives': 0.0...","{'Brown Corpus Baseline Min': 0.000355, 'Brown...",-16.3
4,cd931ee1-01f5-4936-b02f-df6bdc36dba5,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,5,assistant,Thank you. Let’s start. Please note that there...,2024-03-28 14:34:07,ai,144,798,90,0.625000,73.78,"{'Nouns': 0.17575757575757575, 'Verbs': 0.1636...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,92748138-2878-4c89-8995-991e2bf6faa1,e3463372-5289-4a62-b6d0-1fb5b4c1e87d,https://cloud.getliteral.ai/projects/interview...,5,27,assistant,You've highlighted concerns about the lack of ...,2024-03-28 14:54:16,ai,48,329,40,0.833333,38.32,"{'Nouns': 0.2033898305084746, 'Verbs': 0.06779...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",14.1
260,eb993adc-03a3-4632-a8c5-777c6118d208,e3463372-5289-4a62-b6d0-1fb5b4c1e87d,https://cloud.getliteral.ai/projects/interview...,5,28,human,"voting, representation",2024-03-28 14:54:57,ai,2,22,2,1.000000,-91.30,"{'Nouns': 0.6666666666666666, 'Verbs': 0.0, 'A...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",29.0
261,9f7ea78d-7057-4e7d-a13e-3210677b64fc,e3463372-5289-4a62-b6d0-1fb5b4c1e87d,https://cloud.getliteral.ai/projects/interview...,5,29,assistant,Voting and representation stand out as critica...,2024-03-28 14:55:04,ai,58,350,48,0.827586,51.89,"{'Nouns': 0.1875, 'Verbs': 0.109375, 'Adjectiv...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",11.9
262,dbadb15d-a9c5-4da5-886a-6b0ebe66a702,e3463372-5289-4a62-b6d0-1fb5b4c1e87d,https://cloud.getliteral.ai/projects/interview...,5,30,human,doesnt take into account people with disabitli...,2024-03-28 14:55:20,ai,7,50,7,1.000000,30.53,"{'Nouns': 0.375, 'Verbs': 0.125, 'Adjectives':...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",11.7


In [ ]:
def calculate_rare_words(text):
    tokens = word_tokenize(text.lower())
    brown_freqs = [word_frequency(token, 'en', wordlist='large', minimum=0.0) for token in tokens]
    brown_min = np.min(brown_freqs)
    brown_mean = np.mean(brown_freqs)

    return {
        "Brown Corpus Baseline Min": brown_min,
        "Brown Corpus Baseline Mean": brown_mean
    }

In [ ]:
def calculate_rare(df):
    df['Rare Words'] = df['MessageContent'].apply(calculate_rare_words)

    return df

In [ ]:
calculate_rare(df_ai_responses)

,StepID,ThreadID,StepURL,ParticipantID,MessageID,Interlocutor,MessageContent,Timestamp,Interviewer,Word Count,Length,Unique Words,Type-Token-Ratio,Flesch Reading Ease,POS,Rare Words,Automated Readability Index
0,273e7fde-1824-4801-b8dc-8acb7e160671,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,1,assistant,Welcome to our in-depth interview about democr...,2024-03-28 14:32:19,ai,19,120,16,0.842105,65.08,"{'Nouns': 0.16666666666666666, 'Verbs': 0.1666...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",7.0
1,6ce3c664-be62-4c39-8a32-beb10dbb4a0f,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,2,human,yes,2024-03-28 14:33:32,ai,1,3,1,1.000000,121.22,"{'Nouns': 0.0, 'Verbs': 0.0, 'Adjectives': 0.0...","{'Brown Corpus Baseline Min': 0.000316, 'Brown...",-6.9
2,241d3de5-e09b-4db3-a769-cbe8269f156a,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,3,assistant,Before we start with the questions on society ...,2024-03-28 14:33:36,ai,24,133,22,0.916667,64.04,"{'Nouns': 0.19230769230769232, 'Verbs': 0.0769...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",12.1
3,660e7b54-8585-45c5-9303-881df97f4562,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,4,human,7,2024-03-28 14:33:54,ai,1,1,1,1.000000,121.22,"{'Nouns': 0.0, 'Verbs': 0.0, 'Adjectives': 0.0...","{'Brown Corpus Baseline Min': 0.000355, 'Brown...",-16.3
4,cd931ee1-01f5-4936-b02f-df6bdc36dba5,3932df7c-fe12-4188-bbc9-9de38d61c7a7,https://cloud.getliteral.ai/projects/interview...,7,5,assistant,Thank you. Let’s start. Please note that there...,2024-03-28 14:34:07,ai,144,798,90,0.625000,73.78,"{'Nouns': 0.17575757575757575, 'Verbs': 0.1636...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",7.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,92748138-2878-4c89-8995-991e2bf6faa1,e3463372-5289-4a62-b6d0-1fb5b4c1e87d,https://cloud.getliteral.ai/projects/interview...,5,27,assistant,You've highlighted concerns about the lack of ...,2024-03-28 14:54:16,ai,48,329,40,0.833333,38.32,"{'Nouns': 0.2033898305084746, 'Verbs': 0.06779...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",14.1
260,eb993adc-03a3-4632-a8c5-777c6118d208,e3463372-5289-4a62-b6d0-1fb5b4c1e87d,https://cloud.getliteral.ai/projects/interview...,5,28,human,"voting, representation",2024-03-28 14:54:57,ai,2,22,2,1.000000,-91.30,"{'Nouns': 0.6666666666666666, 'Verbs': 0.0, 'A...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",29.0
261,9f7ea78d-7057-4e7d-a13e-3210677b64fc,e3463372-5289-4a62-b6d0-1fb5b4c1e87d,https://cloud.getliteral.ai/projects/interview...,5,29,assistant,Voting and representation stand out as critica...,2024-03-28 14:55:04,ai,58,350,48,0.827586,51.89,"{'Nouns': 0.1875, 'Verbs': 0.109375, 'Adjectiv...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",11.9
262,dbadb15d-a9c5-4da5-886a-6b0ebe66a702,e3463372-5289-4a62-b6d0-1fb5b4c1e87d,https://cloud.getliteral.ai/projects/interview...,5,30,human,doesnt take into account people with disabitli...,2024-03-28 14:55:20,ai,7,50,7,1.000000,30.53,"{'Nouns': 0.375, 'Verbs': 0.125, 'Adjectives':...","{'Brown Corpus Baseline Min': 0.0, 'Brown Corp...",11.7


## COMBINING HUMAN AND AI INTERVIEWS

In [ ]:
df_human, df_ai = combined_df, df_ai_responses
dfs = pd.concat([df_ai, df_human], ignore_index=True)
df = dfs[dfs['Interlocutor'].isin(['human', 'Respondent'])]  # filter by responses in AI and HUMAN interview

# 🎇 Analysis and Visualization

## 🦮 helper functions for the descriptive overview

In [ ]:
def table_descriptive_overivew(df, relevant_columns):
    # Dictionary to store results
    results = {}

    # Loop through each var_analysis value
    for var_analysis in var_analysis_list:
        avg_word_count_human = df[df['Interlocutor'] == 'human'][var_analysis].mean()
        avg_word_count_ai = df[df['Interlocutor'] == 'Respondent'][var_analysis].mean()
        std_word_count_human = df[df['Interlocutor'] == 'human'][var_analysis].std()
        std_word_count_ai = df[df['Interlocutor'] == 'Respondent'][var_analysis].std()

        # Store the results in the dictionary
        results[var_analysis] = (avg_word_count_human, avg_word_count_ai, std_word_count_human, std_word_count_ai)

    # Generate LaTeX table
    latex_table = """
    \\documentclass{article}
    \\usepackage{booktabs}
    \\begin{document}

    \\begin{table}[h!]
    \\centering
    \\begin{tabular}{lcccc}
    \\toprule
    Variable & Human & AI \\\\
    \\midrule
    """

    for var_analysis, (human_avg, ai_avg, human_std, ai_std) in results.items():
        latex_table += f"{var_analysis} & {human_avg:.2f} & {ai_avg:.2f} & {human_std:.2f} & {ai_std:.2f} \\\\\n"

    latex_table += """
    \\bottomrule
    \\end{tabular}
    \\caption{Average Values for Human and AI Interlocutors}
    \\label{table:avg_values}
    \\end{table}

    \\end{document}
    """

    # Print the LaTeX table
    return latex_table

## print the latex table for the descriptive overview

In [ ]:
var_analysis_list = ["Word Count", "Length", "Flesch Reading Ease", "Type-Token-Ratio", "Unique Words"]  # decide which columns/vars you want to have in the latex table
print(table_descriptive_overivew(df, var_analysis_list))


    \documentclass{article}
    \usepackage{booktabs}
    \begin{document}

    \begin{table}[h!]
    \centering
    \begin{tabular}{lcccc}
    \toprule
    Variable & Human & AI \\
    \midrule
    Word Count & 32.81 & 52.39 & 38.80 & 49.80 \\
Length & 166.05 & 241.06 & 194.64 & 231.67 \\
Flesch Reading Ease & 62.22 & 77.66 & 46.49 & 25.14 \\
Type-Token-Ratio & 0.87 & 0.76 & 0.14 & 0.18 \\
Unique Words & 23.64 & 32.31 & 21.59 & 25.66 \\

    \bottomrule
    \end{tabular}
    \caption{Average Values for Human and AI Interlocutors}
    \label{table:avg_values}
    \end{table}

    \end{document}
    


## Voice-focused comparison

## 🦮 helper functions to analyze the voice users

In [ ]:
def get_mean_std_ai(df, var_analysis, participant_id_list):
    df_filtered = df[
              (df['Interlocutor'] == 'human') &
              (df['ParticipantID'].isin(participant_id_list))
          ][var_analysis]

    return df_filtered, df_filtered.mean(), df_filtered.std()

def generate_latex_table(value_list):
    data = []
    settings = ['w_voice', 'wo_voice', 'w_voice_poor', 'w_voice_good']

    for i, setting_data in enumerate(value_list):
        for value in setting_data:
            data.append({'Setting': settings[i], 'Value': value})

    df = pd.DataFrame(data)

    # Check for negative values
    if df['Value'].min() < 0:
        print("Warning: The dataset contains negative values.")
        print(f"Minimum value: {df['Value'].min()}")

    latex_code = [
        "\\begin{table}[h]",
        "\\centering",
        "\\begin{tabular}{|l|c|c|c|c|}",
        "\\hline",
        "\\textbf{Setting} & \\textbf{Mean} & \\textbf{Std Dev} & \\textbf{Min} & \\textbf{Max} \\\\",
        "\\hline"
    ]

    for i, setting in enumerate(settings):
        subset = df[df['Setting'] == setting]['Value']
        row = f"{settings[i]} & {subset.mean():.2f} & {subset.std():.2f} & {subset.min():.2f} & {subset.max():.2f} \\\\"
        latex_code.append(row)

    latex_code.extend([
        "\\hline",
        "\\end{tabular}",
        "\\caption{Results from var\\_analysis}",
        "\\label{tab:results}",
        "\\end{table}"
    ])

    return "\n".join(latex_code)

def get_ttest_results(df, ana_var):
    return ttest_ind(df[df['Interlocutor'] == 'human'][ana_var], df[df['Interlocutor'] == 'Respondent'][ana_var])

## print the latex table for the results of the voice-users

Flesch Reading Ease (Readbility) results

In [ ]:
ana_var = 'Flesch Reading Ease'

# we can also check for a significant difference between the select var
print(get_ttest_results(df, ana_var))

# BreakoutRoom Nummer == Participation
value_list = [
    get_mean_std_ai(df, ana_var, [2,3,7])[0], # participants w voice
    get_mean_std_ai(df, ana_var, [1])[0], # participants wo voice
    get_mean_std_ai(df, ana_var, [3,7])[0], # participants w voice poor
    get_mean_std_ai(df, ana_var, [2])[0] # participants w voice good
]

latex_table = generate_latex_table(value_list)
print(latex_table)

TtestResult(statistic=-3.277565894438894, pvalue=0.001194393836638445, df=252.0)
Minimum value: -119.69
\begin{table}[h]
\centering
\begin{tabular}{|l|c|c|c|c|}
\hline
\textbf{Setting} & \textbf{Mean} & \textbf{Std Dev} & \textbf{Min} & \textbf{Max} \\
\hline
w_voice & 48.32 & 55.61 & -119.69 & 121.22 \\
wo_voice & 77.96 & 25.93 & 46.78 & 121.22 \\
w_voice_poor & 54.94 & 43.41 & -36.46 & 121.22 \\
w_voice_good & 37.40 & 71.32 & -119.69 & 119.19 \\
\hline
\end{tabular}
\caption{Results from var\_analysis}
\label{tab:results}
\end{table}


Word Count results

In [ ]:
ana_var = 'Word Count'

# we can also check for a significant difference between the select var
t_stat, p_value = ttest_ind(df[df['Interlocutor'] == 'human'][ana_var], df[df['Interlocutor'] == 'Respondent'][ana_var])
print(t_stat, p_value)

# BreakoutRoom Nummer == Partici
value_list = [
    get_mean_std_ai(df, ana_var, [2,3,7])[0], # participants w voice
    get_mean_std_ai(df, ana_var, [1])[0], # participants wo voice
    get_mean_std_ai(df, ana_var, [3,7])[0], # participants w voice poor
    get_mean_std_ai(df, ana_var, [2])[0] # participants w voice good
]

latex_table = generate_latex_table(value_list)
print(latex_table)

-2.5495521155054375 0.011379293127562161
\begin{table}[h]
\centering
\begin{tabular}{|l|c|c|c|c|}
\hline
\textbf{Setting} & \textbf{Mean} & \textbf{Std Dev} & \textbf{Min} & \textbf{Max} \\
\hline
w_voice & 48.66 & 48.74 & 1.00 & 205.00 \\
wo_voice & 21.47 & 22.09 & 1.00 & 71.00 \\
w_voice_poor & 40.03 & 33.98 & 1.00 & 123.00 \\
w_voice_good & 62.90 & 64.91 & 2.00 & 205.00 \\
\hline
\end{tabular}
\caption{Results from var\_analysis}
\label{tab:results}
\end{table}
